<a href="https://colab.research.google.com/github/sapjunior/chulacv2018/blob/master/Lab%207%20-%20YOLO%20(You%20Only%20Look%20Once).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2110443 - Computer Vision (2018/1)
## Lab 7 - YOLO (You Only Look Once)
In this lab, we will learn how to use a famouse regression based object detection algorithm name YOLO (You Only Look Once). In this class main repository, we already prepare a PyTorch version of YOLO  based on an [original Implementation](https://github.com/pjreddie/darknet), 


In [0]:
# Install based package
!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

# Clone YOLO based code
!git clone https://github.com/sapjunior/objdetection.git
  
# Download trained weight on COCO dataset
!wget https://cgci.cp.eng.chula.ac.th/yolo-darknet53-coco.pth -O yolo-darknet53-coco.pth

The above command will clone [this repository](https://github.com/sapjunior/objdetection) in to objdetection directory. The internal folder is organized as follows:
  - dataset   
  => ListDataset : Read Object Detection Dataset from file. The input text file should be in following format (one line per image file)
  
>>`a.jpg xmin ymin xmax ymax label xmin ymin xmax ymax label`
  
  - utils ==> Utility functions, such as, calculate IOU between two boxes and NMS
  - model  
    ==> backbone : This directory contains a backbone network (feature extractor) for YOLO. In this example we use darknet-53 as a default network.
    ==> yolo.py : A main network for YOLO Object Detection framework
    ![alt text](https://cdn-images-1.medium.com/max/1750/1*d4Eg17IVJ0L41e7CTWLLSg.png/)

### Import OpenCV, PyTorch, Numpy and Matplotlib

In [0]:
import cv2
import torch
import numpy as np
from matplotlib import pyplot as plt
from objdetection.models.yolo import YOLONet
from objdetection.utils.yoloBoxCoder import YOLOBoxCoder, YOLOBoxPostProcess

### Network Parameters ###

In [0]:
cocoNumClass = 80
trainImageSize = (412,412)
anchors = [ [[116, 90], [156, 198], [373, 326]],
            [[30, 61], [62, 45], [59, 119]],
            [[10, 13], [16, 30], [33, 23]]]
with open('objdetection/coco.names') as f:
    fileContent = f.readlines()
cocoClassName = [x.strip() for x in fileContent]
print(cocoClassName)

### Decalare Network and Load Trained Weight

In [0]:
net = YOLONet(numClass = cocoNumClass)
checkpoint = torch.load('yolo-darknet53-coco.pth', map_location=lambda storage, location: storage)
net.load_state_dict(checkpoint)
if torch.cuda.is_available():
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
  net.cuda()
net.eval()
boxCoder = YOLOBoxCoder(anchors=anchors, numClass= cocoNumClass, inputFeatMapSizes=net.outputFeatMapSizes)

### Input Image Preprocess and Warpped Classify Function

In [0]:
def preprocess(inputImage, trainImageSize):
  preprocessImage = cv2.cvtColor(inputImage.copy(), cv2.COLOR_BGR2RGB)
  preprocessImage = cv2.resize(preprocessImage, trainImageSize)
  preprocessImage = (preprocessImage.astype(np.float32) / 255).transpose(2,0,1)
  return preprocessImage

In [0]:
def classify(net, trainImageSize, inputImage):
  preprocessImage = preprocess(inputImage, trainImageSize)
  batchImage = torch.from_numpy(np.array([preprocessImage]))
  if torch.cuda.is_available():
    batchImage = batchImage.cuda()
    
  with torch.no_grad():
    netOutputs = net(batchImage)
    
  # Output is in 412x412 image size => We need to multiple by inputImage resized ratio
  outputBoxes = YOLOBoxPostProcess(boxCoder.decode(netOutputs), numClass = 80)
  
  outputImage = cv2.cvtColor(inputImage.copy(), cv2.COLOR_BGR2RGB)
  resizeRatioH,resizeRatioW = np.array(inputImage.shape[0:2])/np.array(trainImageSize)
  
  outputBoxesRescale = []
  
  
  for x1,y1,x2,y2,conf,cls_pred in outputBoxes[0]:
    
    # Rescale bbox by resized ratio
    x1, y1, x2, y2 = int(x1*resizeRatioW), int(y1*resizeRatioH), int(x2*resizeRatioW), int(y2*resizeRatioH)
    
    # Append result to rescaled output
    outputBoxesRescale.append([x1,y1,x2,y2,conf,cls_pred])
    
    # Draw Result
    cv2.putText(outputImage, cocoClassName[int(cls_pred)]+' : '+str(round(conf,3))
                ,(x1,y1+20), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(255,0,0),2,cv2.LINE_AA)
    cv2.rectangle(outputImage,(x1,y1),(x2,y2),(0,255,0),3)
    
  return outputImage, np.array(outputBoxesRescale)

### Read input image and classify

In [0]:
# Put your image here
inputImage = cv2.imread('objdetection/yolo-test.png')
inputImageRGB = cv2.cvtColor(inputImage, cv2.COLOR_BGR2RGB)
plt.imshow(inputImageRGB)
plt.show()

In [0]:
outputImage, outputBoxes = classify(net, trainImageSize, inputImage)
plt.imshow(outputImage)
plt.show()